# Dimensão Operador

Este notebook carrega e atualiza a dimensão de operadores do call center

###Execução

In [3]:
#deleta o arquivo de operador
dbutils.fs.rm('/dados/operators.csv')

Out[1]: True

Download dos dados

In [5]:
import requests
import pyspark.sql.functions as F

#cria um diretório de entrada "dados", onde colocaremos os dados do sistema de origem
dbutils.fs.mkdirs('/dados')

#faz o download dos dados de atendimento
url = 'https://raw.githubusercontent.com/leandrohmvieira/talk-data-lakehouse/master/data/operators.csv'
r = requests.get(url, allow_redirects=True)
open('/dbfs/dados/operators.csv', 'wb').write(r.content)

Out[2]: 205

Criação da dimensão

In [7]:
%sql 
CREATE TABLE IF NOT EXISTS dim_operador (
  id_operador integer,
  nome_operador string,
  turno_operador string,
  produto string
)
USING delta 
OPTIONS
(
  path = '/delta/dim_operador'
)

Carrega a dimensão operador

In [9]:
operador = spark.read.csv('/dados/operators.csv',header=True)

#cadastra os dados de operador no contexto SQL
sqlContext.registerDataFrameAsTable(operador,'updates_operador')

In [10]:
%sql

MERGE INTO dim_operador t
    USING updates_operador s
    ON s.id_operador = t.id_operador
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *

Confere os dados

In [12]:
%sql select * from dim_operador

id_operador,nome_operador,turno_operador,produto
1,Leandro,tarde,cobranca
2,Karina,manha,credito
4,maria,noite,servicos
6,Allan,tarde,cobranca
3,joao,manha,cobranca
5,Denis,noite,credito


## Time travel

In [14]:
%sql DESCRIBE HISTORY dim_operador

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
2,2020-04-22T23:09:13.000+0000,2666043451690755,leandrohmvieira@gmail.com,MERGE,Map(predicate -> (CAST(s.`id_operador` AS INT) = t.`id_operador`)),null,List(3538763988665931),0422-185803-joist158,1,WriteSerializable,false,null
1,2020-04-22T23:06:13.000+0000,2666043451690755,leandrohmvieira@gmail.com,MERGE,Map(predicate -> (CAST(s.`id_operador` AS INT) = t.`id_operador`)),null,List(3538763988665931),0422-185803-joist158,0,WriteSerializable,false,null
0,2020-04-22T23:02:35.000+0000,2666043451690755,leandrohmvieira@gmail.com,CREATE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(3538763988665931),0422-185803-joist158,null,SnapshotIsolation,true,null


In [15]:
%sql select * from dim_operador version as of 1

id_operador,nome_operador,turno_operador,produto
1,Leandro,tarde,cobranca
2,Karina,manha,credito
3,joao,manha,cobranca
4,maria,noite,servicos
5,Denis,manha,credito
